In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSR_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSR_avg%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR_avg%i.log' % k, mode='a')

abtemp_prev = abTempF.curr.copy()
fluxs_prev = FluxS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    abtemp_prev[:] = abTempF.curr
    fluxs_prev[:] = FluxS.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = 0.5*(abTempF.curr[f2s]+abtemp_prev[f2s])
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = 0.5*(FluxS.curr[s2f]+fluxs_prev[s2f])
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(FluxF)
                under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=11.
step=1, pc_iterations=12.
step=2, pc_iterations=12.
step=3, pc_iterations=12.
step=4, pc_iterations=10.
step=5, pc_iterations=10.
step=6, pc_iterations=11.
step=7, pc_iterations=10.
step=8, pc_iterations=10.
step=9, pc_iterations=10.
step=10, pc_iterations=10.
step=11, pc_iterations=10.
step=12, pc_iterations=9.
step=13, pc_iterations=10.
step=14, pc_iterations=9.
step=15, pc_iterations=9.
step=16, pc_iterations=10.
step=17, pc_iterations=10.
step=18, pc_iterations=9.
step=19, pc_iterations=9.
step=20, pc_iterations=9.
step=21, pc_iterations=9.
step=22, pc_iterations=9.
step=23, pc_iterations=9.
step=24, pc_iterations=9.
step=25, pc_iterations=9.
step=26, pc_iterations=9.
step=27, pc_iterations=9.
step=28, pc_iterations=9.
step=29, pc_iterations=9.
step=30, pc_iterations=9.
step=31, pc_iterations=9.
step=32, pc_iterations=9.
step=33, pc_iterations=9.
step=34, pc_iterations=9.
step=35, pc_iterations=9.
step=36, pc_iterations=9.
step=37, pc_iterations=9.
step=38

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSR_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.88732768 0.9284308  0.9407619  0.94018647 0.94136363 0.94372148
 0.94551604 0.94720695 0.94940295 0.95065123 0.95217989 0.95338251
 0.95491941 0.95607462 0.95737531 0.95835965 0.9595841  0.96060952
 0.96152362 0.96258701 0.96343646 0.96437217 0.96522522 0.96599085
 0.96683592 0.96760245 0.96839386 0.96906507 0.96968414 0.97034801
 0.9710009  0.97158853 0.97217653 0.97278564 0.97332103 0.97381657
 0.97436592 0.97486553 0.97538141 0.97582939 0.97625275 0.9766685
 0.97709769 0.97750779 0.97790651 0.97827628 0.9786689  0.97899975
 0.97933116 0.97969712 0.98002797 0.98032488 0.98062463 0.98093881
 0.98119965 0.98148383 0.98173216 0.98198939 0.98225321 0.98247028
 0.98268588 0.98292878 0.98314436 0.98333908 0.98351585 0.98373011
 0.98388436 0.98405855 0.98420452 0.98435192 0.98450272 0.98462165
 0.98473232 0.98486788 0.98497156 0.98506227 0.98513208 0.9852267
 0.98526839 0.98532851 0.98535375 0.9853784  0.98539116 0.98540639
 0.9853875  0.98534511 0.98532108 0.9852471  0.98515341 0.985066